In [1]:
"""
    Copyright (c) 2024 Idiap Research Institute, http://www.idiap.ch/
    Written by Cem Bilaloglu <cem.bilaloglu@idiap.ch>

    This file is part of tactileErgodicExploration.

    tactileErgodicExploration is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License version 3 as
    published by the Free Software Foundation.

    tactileErgodicExploration is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with tactileErgodicExploration. If not, see <http://www.gnu.org/licenses/>.
"""


import numpy as np
import open3d as o3d
import cv2
from hedac_utils import *
from plotting_utils import *

input_img_dir = "images/"
point_cloud_dir = "point_clouds/" # for loading the point cloud

In [2]:
# Select the object and load the point cloud
# ==========================================
obj_name = "bun270"
# obj_name = "cup_X"
# obj_name = "plate_shapes"


filename = f"{point_cloud_dir}{obj_name}.ply"
pcd = o3d.io.read_point_cloud(filename)

# # Rotate the point cloud
# # if you want to project the image afterwards to a different view
# R = pcd.get_rotation_matrix_from_xyz((-0.1, -0.1*np.pi/2,0*np.pi/2))
# pcd.rotate(R, center=(0,0,0))

pcd_tree = o3d.geometry.KDTreeFlann(pcd)
vertices = np.asarray(pcd.points)

Here projecting the images to the point clouds are inspired by Gabriel Peyre's script:
https://nbviewer.org/github/gpeyre/numerical-tours/blob/master/matlab/meshproc_5_pde.ipynb

In [3]:
# Utility functions
# ===================================================================
def create_sphere_point_cloud(radius=1.0, num_points=10000):
    """
    Create a point cloud from a sphere.

    Parameters:
    -----------
    radius: float
        The radius of the sphere.
    num_points: int
        The number of points to generate on the surface of the sphere.

    Returns:
    --------
    o3d.geometry.PointCloud
        The generated point cloud.
    """
    # Generate random points on the surface of a sphere
    phi = np.random.uniform(0, 2 * np.pi, num_points)
    cos_theta = np.random.uniform(-1, 1, num_points)
    sin_theta = np.sqrt(1 - cos_theta**2)

    x = radius * sin_theta * np.cos(phi)
    y = radius * sin_theta * np.sin(phi)
    z = radius * cos_theta

    # Combine the coordinates into a point cloud
    points = np.vstack((x, y, z)).T
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)

    return point_cloud


def create_plane_point_cloud(size=(1.0, 1.0), num_points=1000):
    """
    Create a point cloud from a plane.

    Parameters:
    -----------
    size: Tuple[float, float]
        The size of the plane in the x and y directions.
    num_points: int
        The number of points to generate on the plane.

    Returns:
    --------
    o3d.geometry.PointCloud
        The generated point cloud.
    """
    # Generate random points on the plane
    x = np.random.uniform(-0.5 * size[0], 0.5 * size[0], num_points)
    y = np.random.uniform(-0.5 * size[1], 0.5 * size[1], num_points)
    z = np.zeros(num_points)

    # Combine the coordinates into a point cloud
    points = np.vstack((x, y, z)).T
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)

    return point_cloud


def bilinear_interpolation_2(grid, pos):
    """
    Perform bilinear interpolation on a 2-D grid.

    Parameters:
    -----------
    grid: numpy.ndarray
        A 2D numpy array representing the grid to interpolate on.
    pos: Tuple[float, float]
        A tuple containing the x and y coordinates of the point to
        interpolate at.

    Returns:
    --------
    int
        The interpolated value at the given point.
    """
    x_arr = np.linspace(0, 1, grid.shape[0])
    y_arr = np.linspace(0, 1, grid.shape[1])
    # find the index of the closest smaller pixel after resampling between
    # [0,1]
    for i in range(grid.shape[0]):
        if pos[0] <= x_arr[i]:
            x = i - 1
            break

    # find the index of the closest smaller pixel after resampling between
    # [0,1]
    for j in range(grid.shape[1]):
        if pos[1] <= y_arr[j]:
            y = j - 1
            break


    def border_interpolate(value, size):
        """
        Interpolate the value for the border handling.

        Parameters:
        -----------
        value: int
            The original value.
        size: int
            The size of the dimension.

        Returns:
        --------
        int
            The interpolated value for the border handling.
        """
        return max(0, min(size - 1, value))

    # find the nearest integers by minding the borders
    x0 = border_interpolate(x, grid.shape[1])
    x1 = border_interpolate(x + 1, grid.shape[1])
    y0 = border_interpolate(y, grid.shape[0])
    y1 = border_interpolate(y + 1, grid.shape[0])

    # Distance from lower integers
    xd = pos[0] - x_arr[x0]
    yd = pos[1] - y_arr[y0]

    # Interpolate on x-axis
    c01 = grid[y0, x0] * (1 - xd) + grid[y0, x1] * xd
    c11 = grid[y1, x0] * (1 - xd) + grid[y1, x1] * xd
    # Interpolate on y-axis
    c = c01 * (1 - yd) + c11 * yd
    return int(c)


def map_image2plane(vertices, img_name="X"):
    """
    Map an image to a planar point cloud.

    Parameters:
    -----------
    vertices: numpy.ndarray
        The vertices of the point cloud.
    img_name: str
        The name of the image file to map.

    Returns:
    --------
    None
    """
    # Read and show the input image
    img = cv2.imread(f"{input_img_dir}{img_name}.png", cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))
    cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE, img)
    # plt.imshow(img,cmap='gray')
    # plt.show()

    # Texture mapping of the image (uv parameterization)
    # ===================================================================

    # remove the translation of the point cloud and center around the origin
    mean_vertices = np.mean(vertices, axis=0)

    v = vertices - np.tile(mean_vertices, (vertices.shape[0], 1))

    # compute the uv coordinates for each vertex
    u = (v[:, 0] - np.min(v[:, 0])) / (np.max(v[:, 0]) - np.min(v[:, 0]))
    v = (v[:, 1] - np.min(v[:, 1])) / (np.max(v[:, 1]) - np.min(v[:, 1]))

    u0 = np.zeros(vertices.shape[0], dtype=int)
    # I am trying to imitate MATLAB's interp2() function with input
    # sampling
    for vertex_id in range(vertices.shape[0]):
        u0[vertex_id] = bilinear_interpolation_2(
            img.T, np.array([u[vertex_id], v[vertex_id]])
        )
    return u0


def map_image2sphere(vertices, img_name="X"):
    """
    Map an image to a sphere.

    Parameters:
    -----------
    vertices: numpy.ndarray
        The vertices of the sphere.
    img_name: str
        The name of the image file to map.

    Returns:
    --------
    None
    """
    # Read and show the input image
    img = cv2.imread(f"{input_img_dir}{img_name}.png", cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))
    # plt.imshow(img,cmap='gray')
    # plt.show()

    # Texture mapping of the image (uv parameterization)
    # ===================================================================

    # remove the translation of the mesh and center around the origin
    mean_vertices = np.mean(vertices, axis=0)

    v = vertices - np.tile(mean_vertices, (vertices.shape[0], 1))

    # compute the spherical coordinates for each vertex
    norm_v = np.sqrt(np.sum(v**2, axis=1))
    theta = np.arccos(v[:, 0] / (norm_v)) / np.pi
    phi = (np.arctan2(v[:, 1], v[:, 2]) / np.pi + 1) / 2

    u0 = np.zeros(vertices.shape[0], dtype=int)
    # I am trying to imitate MATLAB's interp2() function with input
    # sampling
    for vertex_id in range(vertices.shape[0]):
        u0[vertex_id] = bilinear_interpolation_2(
            img.T, np.array([theta[vertex_id], phi[vertex_id]])
        )
    return u0

In [4]:
# Set the exploration target and visuzalize the point cloud
#==============================================================================

#"""
# image target used with non-planar shape
img_name = "X"
u0 = map_image2sphere(vertices, img_name)
#"""

""" toggle '#' at the beginning to uncomment the following code
# image target used with plane
img_name = "RLI"
u0 = map_image2plane(vertices, img_name)
#"""

""" toggle '#' at the beginning to uncomment the following code
# Dirac target at a random vertex
img_name = "random"
u0 = np.zeros(param.num_vertices,dtype=int)
target_index = random.sample(range(0, vertices.shape[0]), 1)
u0[target_index] = 255
#"""

""" toggle '#' at the beginning to uncomment the following code
# uniform target
img_name = "uniform"
u0 = np.ones(n,dtype=int)*255
#"""

visualize_point_cloud(vertices, u0)

In [5]:
# Save the point cloud with the target
colors = np.zeros((vertices.shape[0],3),dtype=int)
colors[:,0] = u0
pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.io.write_point_cloud(f"{point_cloud_dir}{obj_name}_{img_name}.ply", pcd)

[Open3D WARNING] Write Ply clamped color value to valid range


True